In [98]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [99]:
train = pd.read_csv("train_ready_for_model.csv")

In [100]:
test = pd.read_csv("test_ready_for_model.csv")

In [101]:
result = test[["Loan_ID"]]

In [102]:
train.columns

Index([u'Loan_ID', u'ApplicantIncome', u'CoapplicantIncome', u'Credit_History',
       u'Dependents', u'Education', u'Gender', u'LoanAmount',
       u'Loan_Amount_Term', u'Loan_Status', u'Married', u'Property_Area',
       u'Self_Employed'],
      dtype='object')

In [103]:
result["Loan_Status"] = train.Loan_Status

In [104]:
result.to_csv("res1.csv",index=False)

In [105]:
target = 'Loan_Status'
IDcol = ['Loan_ID']
from sklearn import model_selection, metrics
from sklearn.metrics import classification_report

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
    
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    kfold = model_selection.KFold(n_splits=10, random_state=7)
    
    #CrossValidation:
    cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=kfold, scoring='accuracy')
    print("10-fold cross validation average accuracy: %.3f" % (cv_train_score.mean()))
    #cv_train_score = np.sqrt(np.abs(cv_train_score))
    
    #Print model report:
    print("\nModel Report")
    print(classification_report(dtrain[target].values, dtrain_predictions))
    #print("Train CV score : ", cv_train_score)
    #print("Train RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
        
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

In [106]:
from sklearn.linear_model import LogisticRegression
predictors = [x for x in train.columns if x not in [target]+IDcol]
print("predictors : ",predictors)
# print predictors
alg1 = LogisticRegression()
modelfit(alg1, train, test, predictors, target, IDcol, 'res2.csv')
##coef1 = pd.Series(alg1.coef_, predictors).sort_values()
##coef1.plot(kind='bar', title='Model Coefficients')

('predictors : ', ['ApplicantIncome', 'CoapplicantIncome', 'Credit_History', 'Dependents', 'Education', 'Gender', 'LoanAmount', 'Loan_Amount_Term', 'Married', 'Property_Area', 'Self_Employed'])
10-fold cross validation average accuracy: 0.806

Model Report
             precision    recall  f1-score   support

        0.0       0.91      0.44      0.59       192
        1.0       0.79      0.98      0.88       422

avg / total       0.83      0.81      0.79       614



In [107]:
final=pd.read_csv('res2.csv')

In [108]:
final.Loan_Status = final.Loan_Status.astype(str)

In [112]:
final.Loan_Status=final.Loan_Status.apply( lambda x: x.replace('1.0','Y'))
final.Loan_Status=final.Loan_Status.apply( lambda x: x.replace('0.0','N'))
#final.head()
final.to_csv("sub.csv",index=False)

In [87]:
from sklearn.neighbors import KNeighborsClassifier

In [88]:
from sklearn.neighbors import KNeighborsClassifier
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
for i in range(1,10):
    alg1 = KNeighborsClassifier(n_neighbors=i)
    modelfit(alg1, train, test, predictors, target, IDcol, 'res3.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

10-fold cross validation average accuracy: 0.575

Model Report
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00       192
        1.0       1.00      1.00      1.00       422

avg / total       1.00      1.00      1.00       614

10-fold cross validation average accuracy: 0.506

Model Report
             precision    recall  f1-score   support

        0.0       0.61      1.00      0.75       192
        1.0       1.00      0.70      0.83       422

avg / total       0.88      0.80      0.80       614

10-fold cross validation average accuracy: 0.603

Model Report
             precision    recall  f1-score   support

        0.0       0.70      0.52      0.59       192
        1.0       0.80      0.90      0.85       422

avg / total       0.77      0.78      0.77       614

10-fold cross validation average accuracy: 0.539

Model Report
             precision    recall  f1-score   support

        0.0       0.59      0.64      0.61       

In [89]:
from sklearn.ensemble import RandomForestClassifier
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = RandomForestClassifier(n_estimators=200,max_depth=15, min_samples_leaf=100)
modelfit(alg1, train, test, predictors, target, IDcol, 'res4.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

10-fold cross validation average accuracy: 0.687

Model Report
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00       192
        1.0       0.69      1.00      0.81       422

avg / total       0.47      0.69      0.56       614



In [90]:
from sklearn.ensemble import AdaBoostClassifier
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = AdaBoostClassifier(n_estimators=200)
modelfit(alg1, train, test, predictors, target, IDcol, 'res5.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

10-fold cross validation average accuracy: 0.777

Model Report
             precision    recall  f1-score   support

        0.0       0.92      0.64      0.76       192
        1.0       0.86      0.98      0.91       422

avg / total       0.88      0.87      0.86       614



In [91]:
from sklearn.tree import DecisionTreeClassifier
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = DecisionTreeClassifier(max_depth=15, min_samples_leaf=100)
modelfit(alg1, train, test, predictors, target, IDcol, 'res6.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

10-fold cross validation average accuracy: 0.687

Model Report
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00       192
        1.0       0.69      1.00      0.81       422

avg / total       0.47      0.69      0.56       614

